In [ ]:
from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d_%H%M")

!ncu \
  --set full \
  --force-overwrite \
  -o colab_ncu_{timestamp} \
  ./wmma_matrixMul2 1024 2048 1024 > output_ncu_{timestamp}.txt

from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d_%H%M")

!ncu \
  --set full \
  --force-overwrite \
  -o colab_ncu_{timestamp} \
  ./wmma_matrixMul4 1024 2048 1024 > output_ncu_{timestamp}.txt

#RUN PROFILING NCU at FULL on A100
from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d_%H%M")

!ncu \
  --set full \
  --force-overwrite \
  -o colab_ncu_{timestamp} \
  ./wmma_matrixMul8 1024 2048 1024 > output_ncu_{timestamp}.txt

In [ ]:
!ncu --metrics sm__pipe_tensor_cycles_active,sm__pipe_shared_cycles_active,l1tex__data_bank_conflicts_pipe_lsu_mem_shared_op_ld,sm__throughput.avg.pct_of_peak_sustained_elapsed ./wmma_matrixMul2 8192 2048 8192
!ncu --metrics sm__pipe_tensor_cycles_active,sm__pipe_shared_cycles_active,l1tex__data_bank_conflicts_pipe_lsu_mem_shared_op_ld,sm__throughput.avg.pct_of_peak_sustained_elapsed ./wmma_matrixMul4 8192 2048 8192
!ncu --metrics sm__pipe_tensor_cycles_active,sm__pipe_shared_cycles_active,l1tex__data_bank_conflicts_pipe_lsu_mem_shared_op_ld,sm__throughput.avg.pct_of_peak_sustained_elapsed ./wmma_matrixMul8 8192 2048 8192

Matrix dimensions: A(8192 x 2048) * B(2048 x 8192) = C(8192 x 8192)


=== WMMA Configuration ===
Fragment dimensions: M=16, N=16, K=16
Number of fragments:
  M direction: 512 fragments (covering 8192 elements)
  N direction: 512 fragments (covering 8192 elements)
  K direction: 128 fragments (covering 2048 elements)
  Total fragments used: 33554432
  Fragment type: half precision (FP16) input, float accumulator

Running CPU reference...
CPU time: 1538583.501 ms

==PROF== Connected to process 849 (/content/wmma_matrixMul2)
Running tiled GEMM kernel...
==PROF== Profiling "gemm" - 0: 0%..
==WARNING== Launching the workload is taking more time than expected. If this continues to hang, terminate the profile and re-try by profiling the range of all related launches using '--replay-mode range'. See https://docs.nvidia.com/nsight-compute/ProfilingGuide/index.html#replay for more details.
..50%....100% - 6 passes
GEMM time: 18730.105 ms
GEMM average relative error: 0.0000000000

==PROF== Profil

In [ ]:
!ncu --metrics sm__pipe_tensor_cycles_active,sm__pipe_shared_cycles_active,l1tex__data_bank_conflicts_pipe_lsu_mem_shared_op_ld,sm__throughput.avg.pct_of_peak_sustained_elapsed ./wmma_matrixMul2 1024 2048 1024
!ncu --metrics sm__pipe_tensor_cycles_active,sm__pipe_shared_cycles_active,l1tex__data_bank_conflicts_pipe_lsu_mem_shared_op_ld,sm__throughput.avg.pct_of_peak_sustained_elapsed ./wmma_matrixMul4 1024 2048 1024
!ncu --metrics sm__pipe_tensor_cycles_active,sm__pipe_shared_cycles_active,l1tex__data_bank_conflicts_pipe_lsu_mem_shared_op_ld,sm__throughput.avg.pct_of_peak_sustained_elapsed ./wmma_matrixMul8 1024 2048 1024

Matrix dimensions: A(1024 x 2048) * B(2048 x 1024) = C(1024 x 1024)


=== WMMA Configuration ===
Fragment dimensions: M=16, N=16, K=16
Number of fragments:
  M direction: 64 fragments (covering 1024 elements)
  N direction: 64 fragments (covering 1024 elements)
  K direction: 128 fragments (covering 2048 elements)
  Total fragments used: 524288
  Fragment type: half precision (FP16) input, float accumulator

Running CPU reference...
CPU time: 18779.145 ms

==PROF== Connected to process 9717 (/content/wmma_matrixMul2)
Running tiled GEMM kernel...
==PROF== Profiling "gemm" - 0: 0%....50%....100% - 6 passes
GEMM time: 764.281 ms
GEMM average relative error: 0.0000000000

==PROF== Profiling "double_to_half" - 1: 0%....50%....100% - 6 passes
==PROF== Profiling "double_to_half" - 2: 0%....50%....100% - 6 passes
Running WMMA Tensor Core kernel...
WMMA Grid: (32, 64), Block: (64, 1)
==PROF== Profiling "wmma_gemm" - 3: 0%....50%....100% - 6 passes
==PROF== Profiling "float_to_double" - 4: 0%...

In [ ]:
import torch

major, minor = torch.cuda.get_device_capability()

arch_flag = f"sm_{major}{minor}"
print(f"Detected Architecture: {arch_flag}")

!nvcc -arch={arch_flag} -O3 -o wmma_matrixMul2 wmma_matrixMul2.cu
!nvcc -arch={arch_flag} -O3 -o wmma_matrixMul4 wmma_matrixMul4.cu
!nvcc -arch={arch_flag} -O3 -o wmma_matrixMul8 wmma_matrixMul8.cu

Detected Architecture: sm_75


In [ ]:
%%writefile wmma_matrixMul2.cu

#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <cuda_fp16.h>
#include <mma.h>
#include <chrono>
#include <cmath>

using namespace nvcuda;

#define CHECK(call) do {                                 \
    cudaError_t err = (call);                            \
    if (err != cudaSuccess) {                            \
        fprintf(stderr, "CUDA error: %s (%s:%d)\n",      \
                cudaGetErrorString(err), __FILE__, __LINE__); \
        exit(1);                                         \
    }                                                    \
} while (0)

// WMMA tile dimensions (for half precision: 16x16x16)
#define WMMA_M 16
#define WMMA_N 16
#define WMMA_K 16

// Tiled GEMM kernel (baseline GPU implementation)
#define TILE_SIZE 16

__global__ void gemm(const double *A, const double *B, double *C,
                     int M, int K, int N) {
    __shared__ double As[TILE_SIZE][TILE_SIZE];
    __shared__ double Bs[TILE_SIZE][TILE_SIZE];

    int row = blockIdx.y * TILE_SIZE + threadIdx.y;
    int col = blockIdx.x * TILE_SIZE + threadIdx.x;

    double sum = 0.0;

    for (int t = 0; t < (K + TILE_SIZE - 1) / TILE_SIZE; t++) {
        // Load tiles into shared memory
        if (row < M && t * TILE_SIZE + threadIdx.x < K)
            As[threadIdx.y][threadIdx.x] = A[row * K + t * TILE_SIZE + threadIdx.x];
        else
            As[threadIdx.y][threadIdx.x] = 0.0;

        if (col < N && t * TILE_SIZE + threadIdx.y < K)
            Bs[threadIdx.y][threadIdx.x] = B[(t * TILE_SIZE + threadIdx.y) * N + col];
        else
            Bs[threadIdx.y][threadIdx.x] = 0.0;

        __syncthreads();

        // Compute partial product
        for (int k = 0; k < TILE_SIZE; k++) {
            sum += As[threadIdx.y][k] * Bs[k][threadIdx.x];
        }
        __syncthreads();
    }

    if (row < M && col < N) {
        C[row * N + col] = sum;
    }
}

// Convert double precision to half precision
__global__ void double_to_half(const double *input, half *output, int size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < size) {
        output[idx] = __float2half((float)input[idx]);
    }
}

// Convert half precision to double precision
__global__ void half_to_double(const half *input, double *output, int size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < size) {
        output[idx] = (double)__half2float(input[idx]);
    }
}

// WMMA kernel for matrix multiplication using Tensor Cores
// C (M x N) = A (M x K) * B (K x N)
// Using half precision for input and accumulation in float
// Output is float (will be converted to double later if needed)
__global__ void wmma_gemm(const half *A, const half *B, float *C,
                          int M, int K, int N) {
    // Leading dimensions
    int lda = K;
    int ldb = N;
    int ldc = N;

    // Each warp computes one 16x16 output tile
    // Warp ID within the grid (each warp is an independent unit)
    int warpM = (blockIdx.x * blockDim.x + threadIdx.x) / 32;
    int warpN = (blockIdx.y * blockDim.y + threadIdx.y);

    // Calculate the starting row and column for this warp's output tile
    int warp_row = warpM * WMMA_M;
    int warp_col = warpN * WMMA_N;

    // Bounds check
    if (warp_row >= M || warp_col >= N)
        return;

    // Declare the fragments
    wmma::fragment<wmma::matrix_a, WMMA_M, WMMA_N, WMMA_K, half, wmma::row_major> a_frag;
    wmma::fragment<wmma::matrix_b, WMMA_M, WMMA_N, WMMA_K, half, wmma::row_major> b_frag;
    wmma::fragment<wmma::accumulator, WMMA_M, WMMA_N, WMMA_K, float> acc_frag;

    // Initialize accumulator to zero
    wmma::fill_fragment(acc_frag, 0.0f);

    // Loop over K dimension in steps of WMMA_K
    for (int i = 0; i < K; i += WMMA_K) {
        if (i < K) {
            // Step 1: Load the inputs into fragments
            wmma::load_matrix_sync(a_frag, A + warp_row * lda + i, lda);
            wmma::load_matrix_sync(b_frag, B + i * ldb + warp_col, ldb);

            // Step 2: Perform the matrix multiplication
            wmma::mma_sync(acc_frag, a_frag, b_frag, acc_frag);
        }
    }

    // Step 3: Store the output
    wmma::store_matrix_sync(C + warp_row * ldc + warp_col, acc_frag, ldc, wmma::mem_row_major);
}

// Convert float to double
__global__ void float_to_double(const float *input, double *output, int size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < size) {
        output[idx] = (double)input[idx];
    }
}

// CPU reference implementation
void cpu_gemm(const double *A, const double *B, double *C, int M, int K, int N) {
    for (int i = 0; i < M; i++) {
        for (int j = 0; j < N; j++) {
            double sum = 0.0;
            for (int k = 0; k < K; k++) {
                sum += A[i * K + k] * B[k * N + j];
            }
            C[i * N + j] = sum;
        }
    }
}

// Calculate relative error
double calculate_error(const double *ref, const double *result, int size) {
    double max_error = 0.0;
    double sum_error = 0.0;

    for (int i = 0; i < size; i++) {
        double error = fabs(ref[i] - result[i]);
        double rel_error = error / (fabs(ref[i]) + 1e-10);
        max_error = fmax(max_error, rel_error);
        sum_error += rel_error;
    }

    return sum_error / size;
}

void print_wmma_info(int M, int K, int N) {
    printf("\n=== WMMA Configuration ===\n");
    printf("Fragment dimensions: M=%d, N=%d, K=%d\n", WMMA_M, WMMA_N, WMMA_K);

    // Calculate number of fragments needed
    int frags_M = (M + WMMA_M - 1) / WMMA_M;
    int frags_N = (N + WMMA_N - 1) / WMMA_N;
    int frags_K = (K + WMMA_K - 1) / WMMA_K;

    printf("Number of fragments:\n");
    printf("  M direction: %d fragments (covering %d elements)\n", frags_M, M);
    printf("  N direction: %d fragments (covering %d elements)\n", frags_N, N);
    printf("  K direction: %d fragments (covering %d elements)\n", frags_K, K);
    printf("  Total fragments used: %d\n", frags_M * frags_N * frags_K);
    printf("  Fragment type: half precision (FP16) input, float accumulator\n");
    printf("==========================\n\n");
}

int main(int argc, char **argv) {
    if (argc < 2) {
        printf("Usage: %s <size> or %s <M> <K> <N>\n", argv[0], argv[0]);
        printf("Example: %s 1024 2048 1024  (for A: 1024x2048, B: 2048x1024)\n", argv[0]);
        return 1;
    }

    int M, K, N;
    if (argc == 2) {
        // Square matrices
        M = K = N = atoi(argv[1]);
    } else if (argc == 4) {
        M = atoi(argv[1]);
        K = atoi(argv[2]);
        N = atoi(argv[3]);
    } else {
        fprintf(stderr, "Invalid arguments\n");
        return 1;
    }

    printf("Matrix dimensions: A(%d x %d) * B(%d x %d) = C(%d x %d)\n\n",
           M, K, K, N, M, N);

    // Print WMMA information
    print_wmma_info(M, K, N);

    // Allocate host memory
    size_t bytes_A = M * K * sizeof(double);
    size_t bytes_B = K * N * sizeof(double);
    size_t bytes_C = M * N * sizeof(double);

    double *h_A = (double*)malloc(bytes_A);
    double *h_B = (double*)malloc(bytes_B);
    double *h_C_cpu = (double*)malloc(bytes_C);
    double *h_C_gemm = (double*)malloc(bytes_C);
    double *h_C_wmma = (double*)malloc(bytes_C);

    // Initialize matrices
    for (int i = 0; i < M * K; i++) {
        h_A[i] = (double)(rand() % 100) / 10.0;
    }
    for (int i = 0; i < K * N; i++) {
        h_B[i] = (double)(rand() % 100) / 10.0;
    }

    // CPU computation
    printf("Running CPU reference...\n");
    auto cpu_start = std::chrono::high_resolution_clock::now();
    cpu_gemm(h_A, h_B, h_C_cpu, M, K, N);
    auto cpu_end = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> cpu_duration = cpu_end - cpu_start;
    printf("CPU time: %.3f ms\n\n", cpu_duration.count() * 1000.0);

    // Allocate device memory for double precision
    double *d_A, *d_B, *d_C_gemm;
    CHECK(cudaMalloc(&d_A, bytes_A));
    CHECK(cudaMalloc(&d_B, bytes_B));
    CHECK(cudaMalloc(&d_C_gemm, bytes_C));

    CHECK(cudaMemcpy(d_A, h_A, bytes_A, cudaMemcpyHostToDevice));
    CHECK(cudaMemcpy(d_B, h_B, bytes_B, cudaMemcpyHostToDevice));

    // Run tiled GEMM kernel
    printf("Running tiled GEMM kernel...\n");
    dim3 block_gemm(TILE_SIZE, TILE_SIZE);
    dim3 grid_gemm((N + TILE_SIZE - 1) / TILE_SIZE,
                   (M + TILE_SIZE - 1) / TILE_SIZE);

    cudaEvent_t start, stop;
    CHECK(cudaEventCreate(&start));
    CHECK(cudaEventCreate(&stop));

    CHECK(cudaEventRecord(start));
    gemm<<<grid_gemm, block_gemm>>>(d_A, d_B, d_C_gemm, M, K, N);
    CHECK(cudaEventRecord(stop));
    CHECK(cudaEventSynchronize(stop));

    float gemm_time = 0;
    CHECK(cudaEventElapsedTime(&gemm_time, start, stop));
    CHECK(cudaMemcpy(h_C_gemm, d_C_gemm, bytes_C, cudaMemcpyDeviceToHost));

    double gemm_error = calculate_error(h_C_cpu, h_C_gemm, M * N);
    printf("GEMM time: %.3f ms\n", gemm_time);
    printf("GEMM average relative error: %.10f\n\n", gemm_error);

    // Allocate device memory for half precision (for WMMA)
    half *d_A_half, *d_B_half;
    float *d_C_wmma_float;  // WMMA outputs float
    double *d_C_wmma_double;  // For final conversion to double
    CHECK(cudaMalloc(&d_A_half, M * K * sizeof(half)));
    CHECK(cudaMalloc(&d_B_half, K * N * sizeof(half)));
    CHECK(cudaMalloc(&d_C_wmma_float, M * N * sizeof(float)));
    CHECK(cudaMalloc(&d_C_wmma_double, bytes_C));

    // Convert double to half
    int threads = 256;
    int blocks_A = (M * K + threads - 1) / threads;
    int blocks_B = (K * N + threads - 1) / threads;

    double_to_half<<<blocks_A, threads>>>(d_A, d_A_half, M * K);
    double_to_half<<<blocks_B, threads>>>(d_B, d_B_half, K * N);
    CHECK(cudaDeviceSynchronize());

    // Run WMMA kernel
    printf("Running WMMA Tensor Core kernel...\n");

    // For WMMA: each warp (32 threads) computes one 16x16 output tile
    // Use 128 threads per block (4 warps) for good occupancy
    // Block dimensions: 128 threads in X (4 warps), 1 in Y
    int warps_per_block_x = 2;  // 4 warps per block
    dim3 block_wmma(warps_per_block_x * 32, 1);  // 128 threads in X, 1 in Y
    dim3 grid_wmma(((M + WMMA_M - 1) / WMMA_M + warps_per_block_x - 1) / warps_per_block_x,
                   (N + WMMA_N - 1) / WMMA_N);

    printf("WMMA Grid: (%d, %d), Block: (%d, %d)\n",
           grid_wmma.x, grid_wmma.y, block_wmma.x, block_wmma.y);

    CHECK(cudaEventRecord(start));
    wmma_gemm<<<grid_wmma, block_wmma>>>(d_A_half, d_B_half, d_C_wmma_float, M, K, N);
    CHECK(cudaGetLastError());

    // Convert float output to double for comparison
    int blocks_C = (M * N + threads - 1) / threads;
    float_to_double<<<blocks_C, threads>>>(d_C_wmma_float, d_C_wmma_double, M * N);
    CHECK(cudaGetLastError());

    CHECK(cudaEventRecord(stop));
    CHECK(cudaEventSynchronize(stop));

    float wmma_time = 0;
    CHECK(cudaEventElapsedTime(&wmma_time, start, stop));
    CHECK(cudaMemcpy(h_C_wmma, d_C_wmma_double, bytes_C, cudaMemcpyDeviceToHost));

    double wmma_error = calculate_error(h_C_cpu, h_C_wmma, M * N);
    printf("WMMA time: %.3f ms\n", wmma_time);
    printf("WMMA average relative error: %.10f\n", wmma_error);

    // Debug: Print first few values
    printf("DEBUG - First 10 values:\n");
    printf("  CPU:  ");
    for (int i = 0; i < 10 && i < M * N; i++) printf("%.2f ", h_C_cpu[i]);
    printf("\n  GEMM: ");
    for (int i = 0; i < 10 && i < M * N; i++) printf("%.2f ", h_C_gemm[i]);
    printf("\n  WMMA: ");
    for (int i = 0; i < 10 && i < M * N; i++) printf("%.2f ", h_C_wmma[i]);
    printf("\n\n");

    // Print performance comparison
    printf("=== Performance Summary ===\n");
    printf("CPU:  %.3f ms\n", cpu_duration.count() * 1000.0);
    printf("GEMM: %.3f ms (%.2fx speedup over CPU)\n",
           gemm_time, cpu_duration.count() * 1000.0 / gemm_time);
    printf("WMMA: %.3f ms (%.2fx speedup over CPU, %.2fx over GEMM)\n",
           wmma_time, cpu_duration.count() * 1000.0 / wmma_time, gemm_time / wmma_time);
    printf("\n=== Accuracy Summary ===\n");
    printf("GEMM avg relative error: %.10f\n", gemm_error);
    printf("WMMA avg relative error: %.10f\n", wmma_error);

    // Cleanup
    CHECK(cudaEventDestroy(start));
    CHECK(cudaEventDestroy(stop));
    free(h_A);
    free(h_B);
    free(h_C_cpu);
    free(h_C_gemm);
    free(h_C_wmma);
    CHECK(cudaFree(d_A));
    CHECK(cudaFree(d_B));
    CHECK(cudaFree(d_C_gemm));
    CHECK(cudaFree(d_A_half));
    CHECK(cudaFree(d_B_half));
    CHECK(cudaFree(d_C_wmma_float));
    CHECK(cudaFree(d_C_wmma_double));

    return 0;
}


Writing wmma_matrixMul2.cu


In [ ]:
%%writefile wmma_matrixMul4.cu

#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <cuda_fp16.h>
#include <mma.h>
#include <chrono>
#include <cmath>

using namespace nvcuda;

#define CHECK(call) do {                                 \
    cudaError_t err = (call);                            \
    if (err != cudaSuccess) {                            \
        fprintf(stderr, "CUDA error: %s (%s:%d)\n",      \
                cudaGetErrorString(err), __FILE__, __LINE__); \
        exit(1);                                         \
    }                                                    \
} while (0)

// WMMA tile dimensions (for half precision: 16x16x16)
#define WMMA_M 16
#define WMMA_N 16
#define WMMA_K 16

// Tiled GEMM kernel (baseline GPU implementation)
#define TILE_SIZE 16

__global__ void gemm(const double *A, const double *B, double *C,
                     int M, int K, int N) {
    __shared__ double As[TILE_SIZE][TILE_SIZE];
    __shared__ double Bs[TILE_SIZE][TILE_SIZE];

    int row = blockIdx.y * TILE_SIZE + threadIdx.y;
    int col = blockIdx.x * TILE_SIZE + threadIdx.x;

    double sum = 0.0;

    for (int t = 0; t < (K + TILE_SIZE - 1) / TILE_SIZE; t++) {
        // Load tiles into shared memory
        if (row < M && t * TILE_SIZE + threadIdx.x < K)
            As[threadIdx.y][threadIdx.x] = A[row * K + t * TILE_SIZE + threadIdx.x];
        else
            As[threadIdx.y][threadIdx.x] = 0.0;

        if (col < N && t * TILE_SIZE + threadIdx.y < K)
            Bs[threadIdx.y][threadIdx.x] = B[(t * TILE_SIZE + threadIdx.y) * N + col];
        else
            Bs[threadIdx.y][threadIdx.x] = 0.0;

        __syncthreads();

        // Compute partial product
        for (int k = 0; k < TILE_SIZE; k++) {
            sum += As[threadIdx.y][k] * Bs[k][threadIdx.x];
        }
        __syncthreads();
    }

    if (row < M && col < N) {
        C[row * N + col] = sum;
    }
}

// Convert double precision to half precision
__global__ void double_to_half(const double *input, half *output, int size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < size) {
        output[idx] = __float2half((float)input[idx]);
    }
}

// Convert half precision to double precision
__global__ void half_to_double(const half *input, double *output, int size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < size) {
        output[idx] = (double)__half2float(input[idx]);
    }
}

// WMMA kernel for matrix multiplication using Tensor Cores
// C (M x N) = A (M x K) * B (K x N)
// Using half precision for input and accumulation in float
// Output is float (will be converted to double later if needed)
__global__ void wmma_gemm(const half *A, const half *B, float *C,
                          int M, int K, int N) {
    // Leading dimensions
    int lda = K;
    int ldb = N;
    int ldc = N;

    // Each warp computes one 16x16 output tile
    // Warp ID within the grid (each warp is an independent unit)
    int warpM = (blockIdx.x * blockDim.x + threadIdx.x) / 32;
    int warpN = (blockIdx.y * blockDim.y + threadIdx.y);

    // Calculate the starting row and column for this warp's output tile
    int warp_row = warpM * WMMA_M;
    int warp_col = warpN * WMMA_N;

    // Bounds check
    if (warp_row >= M || warp_col >= N)
        return;

    // Declare the fragments
    wmma::fragment<wmma::matrix_a, WMMA_M, WMMA_N, WMMA_K, half, wmma::row_major> a_frag;
    wmma::fragment<wmma::matrix_b, WMMA_M, WMMA_N, WMMA_K, half, wmma::row_major> b_frag;
    wmma::fragment<wmma::accumulator, WMMA_M, WMMA_N, WMMA_K, float> acc_frag;

    // Initialize accumulator to zero
    wmma::fill_fragment(acc_frag, 0.0f);

    // Loop over K dimension in steps of WMMA_K
    for (int i = 0; i < K; i += WMMA_K) {
        if (i < K) {
            // Step 1: Load the inputs into fragments
            wmma::load_matrix_sync(a_frag, A + warp_row * lda + i, lda);
            wmma::load_matrix_sync(b_frag, B + i * ldb + warp_col, ldb);

            // Step 2: Perform the matrix multiplication
            wmma::mma_sync(acc_frag, a_frag, b_frag, acc_frag);
        }
    }

    // Step 3: Store the output
    wmma::store_matrix_sync(C + warp_row * ldc + warp_col, acc_frag, ldc, wmma::mem_row_major);
}

// Convert float to double
__global__ void float_to_double(const float *input, double *output, int size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < size) {
        output[idx] = (double)input[idx];
    }
}

// CPU reference implementation
void cpu_gemm(const double *A, const double *B, double *C, int M, int K, int N) {
    for (int i = 0; i < M; i++) {
        for (int j = 0; j < N; j++) {
            double sum = 0.0;
            for (int k = 0; k < K; k++) {
                sum += A[i * K + k] * B[k * N + j];
            }
            C[i * N + j] = sum;
        }
    }
}

// Calculate relative error
double calculate_error(const double *ref, const double *result, int size) {
    double max_error = 0.0;
    double sum_error = 0.0;

    for (int i = 0; i < size; i++) {
        double error = fabs(ref[i] - result[i]);
        double rel_error = error / (fabs(ref[i]) + 1e-10);
        max_error = fmax(max_error, rel_error);
        sum_error += rel_error;
    }

    return sum_error / size;
}

void print_wmma_info(int M, int K, int N) {
    printf("\n=== WMMA Configuration ===\n");
    printf("Fragment dimensions: M=%d, N=%d, K=%d\n", WMMA_M, WMMA_N, WMMA_K);

    // Calculate number of fragments needed
    int frags_M = (M + WMMA_M - 1) / WMMA_M;
    int frags_N = (N + WMMA_N - 1) / WMMA_N;
    int frags_K = (K + WMMA_K - 1) / WMMA_K;

    printf("Number of fragments:\n");
    printf("  M direction: %d fragments (covering %d elements)\n", frags_M, M);
    printf("  N direction: %d fragments (covering %d elements)\n", frags_N, N);
    printf("  K direction: %d fragments (covering %d elements)\n", frags_K, K);
    printf("  Total fragments used: %d\n", frags_M * frags_N * frags_K);
    printf("  Fragment type: half precision (FP16) input, float accumulator\n");
    printf("==========================\n\n");
}

int main(int argc, char **argv) {
    if (argc < 2) {
        printf("Usage: %s <size> or %s <M> <K> <N>\n", argv[0], argv[0]);
        printf("Example: %s 1024 2048 1024  (for A: 1024x2048, B: 2048x1024)\n", argv[0]);
        return 1;
    }

    int M, K, N;
    if (argc == 2) {
        // Square matrices
        M = K = N = atoi(argv[1]);
    } else if (argc == 4) {
        M = atoi(argv[1]);
        K = atoi(argv[2]);
        N = atoi(argv[3]);
    } else {
        fprintf(stderr, "Invalid arguments\n");
        return 1;
    }

    printf("Matrix dimensions: A(%d x %d) * B(%d x %d) = C(%d x %d)\n\n",
           M, K, K, N, M, N);

    // Print WMMA information
    print_wmma_info(M, K, N);

    // Allocate host memory
    size_t bytes_A = M * K * sizeof(double);
    size_t bytes_B = K * N * sizeof(double);
    size_t bytes_C = M * N * sizeof(double);

    double *h_A = (double*)malloc(bytes_A);
    double *h_B = (double*)malloc(bytes_B);
    double *h_C_cpu = (double*)malloc(bytes_C);
    double *h_C_gemm = (double*)malloc(bytes_C);
    double *h_C_wmma = (double*)malloc(bytes_C);

    // Initialize matrices
    for (int i = 0; i < M * K; i++) {
        h_A[i] = (double)(rand() % 100) / 10.0;
    }
    for (int i = 0; i < K * N; i++) {
        h_B[i] = (double)(rand() % 100) / 10.0;
    }

    // CPU computation
    printf("Running CPU reference...\n");
    auto cpu_start = std::chrono::high_resolution_clock::now();
    cpu_gemm(h_A, h_B, h_C_cpu, M, K, N);
    auto cpu_end = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> cpu_duration = cpu_end - cpu_start;
    printf("CPU time: %.3f ms\n\n", cpu_duration.count() * 1000.0);

    // Allocate device memory for double precision
    double *d_A, *d_B, *d_C_gemm;
    CHECK(cudaMalloc(&d_A, bytes_A));
    CHECK(cudaMalloc(&d_B, bytes_B));
    CHECK(cudaMalloc(&d_C_gemm, bytes_C));

    CHECK(cudaMemcpy(d_A, h_A, bytes_A, cudaMemcpyHostToDevice));
    CHECK(cudaMemcpy(d_B, h_B, bytes_B, cudaMemcpyHostToDevice));

    // Run tiled GEMM kernel
    printf("Running tiled GEMM kernel...\n");
    dim3 block_gemm(TILE_SIZE, TILE_SIZE);
    dim3 grid_gemm((N + TILE_SIZE - 1) / TILE_SIZE,
                   (M + TILE_SIZE - 1) / TILE_SIZE);

    cudaEvent_t start, stop;
    CHECK(cudaEventCreate(&start));
    CHECK(cudaEventCreate(&stop));

    CHECK(cudaEventRecord(start));
    gemm<<<grid_gemm, block_gemm>>>(d_A, d_B, d_C_gemm, M, K, N);
    CHECK(cudaEventRecord(stop));
    CHECK(cudaEventSynchronize(stop));

    float gemm_time = 0;
    CHECK(cudaEventElapsedTime(&gemm_time, start, stop));
    CHECK(cudaMemcpy(h_C_gemm, d_C_gemm, bytes_C, cudaMemcpyDeviceToHost));

    double gemm_error = calculate_error(h_C_cpu, h_C_gemm, M * N);
    printf("GEMM time: %.3f ms\n", gemm_time);
    printf("GEMM average relative error: %.10f\n\n", gemm_error);

    // Allocate device memory for half precision (for WMMA)
    half *d_A_half, *d_B_half;
    float *d_C_wmma_float;  // WMMA outputs float
    double *d_C_wmma_double;  // For final conversion to double
    CHECK(cudaMalloc(&d_A_half, M * K * sizeof(half)));
    CHECK(cudaMalloc(&d_B_half, K * N * sizeof(half)));
    CHECK(cudaMalloc(&d_C_wmma_float, M * N * sizeof(float)));
    CHECK(cudaMalloc(&d_C_wmma_double, bytes_C));

    // Convert double to half
    int threads = 256;
    int blocks_A = (M * K + threads - 1) / threads;
    int blocks_B = (K * N + threads - 1) / threads;

    double_to_half<<<blocks_A, threads>>>(d_A, d_A_half, M * K);
    double_to_half<<<blocks_B, threads>>>(d_B, d_B_half, K * N);
    CHECK(cudaDeviceSynchronize());

    // Run WMMA kernel
    printf("Running WMMA Tensor Core kernel...\n");

    // For WMMA: each warp (32 threads) computes one 16x16 output tile
    // Use 128 threads per block (4 warps) for good occupancy
    // Block dimensions: 128 threads in X (4 warps), 1 in Y
    int warps_per_block_x = 4;  // 4 warps per block
    dim3 block_wmma(warps_per_block_x * 32, 1);  // 128 threads in X, 1 in Y
    dim3 grid_wmma(((M + WMMA_M - 1) / WMMA_M + warps_per_block_x - 1) / warps_per_block_x,
                   (N + WMMA_N - 1) / WMMA_N);

    printf("WMMA Grid: (%d, %d), Block: (%d, %d)\n",
           grid_wmma.x, grid_wmma.y, block_wmma.x, block_wmma.y);

    CHECK(cudaEventRecord(start));
    wmma_gemm<<<grid_wmma, block_wmma>>>(d_A_half, d_B_half, d_C_wmma_float, M, K, N);
    CHECK(cudaGetLastError());

    // Convert float output to double for comparison
    int blocks_C = (M * N + threads - 1) / threads;
    float_to_double<<<blocks_C, threads>>>(d_C_wmma_float, d_C_wmma_double, M * N);
    CHECK(cudaGetLastError());

    CHECK(cudaEventRecord(stop));
    CHECK(cudaEventSynchronize(stop));

    float wmma_time = 0;
    CHECK(cudaEventElapsedTime(&wmma_time, start, stop));
    CHECK(cudaMemcpy(h_C_wmma, d_C_wmma_double, bytes_C, cudaMemcpyDeviceToHost));

    double wmma_error = calculate_error(h_C_cpu, h_C_wmma, M * N);
    printf("WMMA time: %.3f ms\n", wmma_time);
    printf("WMMA average relative error: %.10f\n", wmma_error);

    // Debug: Print first few values
    printf("DEBUG - First 10 values:\n");
    printf("  CPU:  ");
    for (int i = 0; i < 10 && i < M * N; i++) printf("%.2f ", h_C_cpu[i]);
    printf("\n  GEMM: ");
    for (int i = 0; i < 10 && i < M * N; i++) printf("%.2f ", h_C_gemm[i]);
    printf("\n  WMMA: ");
    for (int i = 0; i < 10 && i < M * N; i++) printf("%.2f ", h_C_wmma[i]);
    printf("\n\n");

    // Print performance comparison
    printf("=== Performance Summary ===\n");
    printf("CPU:  %.3f ms\n", cpu_duration.count() * 1000.0);
    printf("GEMM: %.3f ms (%.2fx speedup over CPU)\n",
           gemm_time, cpu_duration.count() * 1000.0 / gemm_time);
    printf("WMMA: %.3f ms (%.2fx speedup over CPU, %.2fx over GEMM)\n",
           wmma_time, cpu_duration.count() * 1000.0 / wmma_time, gemm_time / wmma_time);
    printf("\n=== Accuracy Summary ===\n");
    printf("GEMM avg relative error: %.10f\n", gemm_error);
    printf("WMMA avg relative error: %.10f\n", wmma_error);

    // Cleanup
    CHECK(cudaEventDestroy(start));
    CHECK(cudaEventDestroy(stop));
    free(h_A);
    free(h_B);
    free(h_C_cpu);
    free(h_C_gemm);
    free(h_C_wmma);
    CHECK(cudaFree(d_A));
    CHECK(cudaFree(d_B));
    CHECK(cudaFree(d_C_gemm));
    CHECK(cudaFree(d_A_half));
    CHECK(cudaFree(d_B_half));
    CHECK(cudaFree(d_C_wmma_float));
    CHECK(cudaFree(d_C_wmma_double));

    return 0;
}


Writing wmma_matrixMul4.cu


In [ ]:
%%writefile wmma_matrixMul8.cu

#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <cuda_fp16.h>
#include <mma.h>
#include <chrono>
#include <cmath>

using namespace nvcuda;

#define CHECK(call) do {                                 \
    cudaError_t err = (call);                            \
    if (err != cudaSuccess) {                            \
        fprintf(stderr, "CUDA error: %s (%s:%d)\n",      \
                cudaGetErrorString(err), __FILE__, __LINE__); \
        exit(1);                                         \
    }                                                    \
} while (0)

// WMMA tile dimensions (for half precision: 16x16x16)
#define WMMA_M 16
#define WMMA_N 16
#define WMMA_K 16

// Tiled GEMM kernel (baseline GPU implementation)
#define TILE_SIZE 16

__global__ void gemm(const double *A, const double *B, double *C,
                     int M, int K, int N) {
    __shared__ double As[TILE_SIZE][TILE_SIZE];
    __shared__ double Bs[TILE_SIZE][TILE_SIZE];

    int row = blockIdx.y * TILE_SIZE + threadIdx.y;
    int col = blockIdx.x * TILE_SIZE + threadIdx.x;

    double sum = 0.0;

    for (int t = 0; t < (K + TILE_SIZE - 1) / TILE_SIZE; t++) {
        // Load tiles into shared memory
        if (row < M && t * TILE_SIZE + threadIdx.x < K)
            As[threadIdx.y][threadIdx.x] = A[row * K + t * TILE_SIZE + threadIdx.x];
        else
            As[threadIdx.y][threadIdx.x] = 0.0;

        if (col < N && t * TILE_SIZE + threadIdx.y < K)
            Bs[threadIdx.y][threadIdx.x] = B[(t * TILE_SIZE + threadIdx.y) * N + col];
        else
            Bs[threadIdx.y][threadIdx.x] = 0.0;

        __syncthreads();

        // Compute partial product
        for (int k = 0; k < TILE_SIZE; k++) {
            sum += As[threadIdx.y][k] * Bs[k][threadIdx.x];
        }
        __syncthreads();
    }

    if (row < M && col < N) {
        C[row * N + col] = sum;
    }
}

// Convert double precision to half precision
__global__ void double_to_half(const double *input, half *output, int size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < size) {
        output[idx] = __float2half((float)input[idx]);
    }
}

// Convert half precision to double precision
__global__ void half_to_double(const half *input, double *output, int size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < size) {
        output[idx] = (double)__half2float(input[idx]);
    }
}

// WMMA kernel for matrix multiplication using Tensor Cores
// C (M x N) = A (M x K) * B (K x N)
// Using half precision for input and accumulation in float
// Output is float (will be converted to double later if needed)
__global__ void wmma_gemm(const half *A, const half *B, float *C,
                          int M, int K, int N) {
    // Leading dimensions
    int lda = K;
    int ldb = N;
    int ldc = N;

    // Each warp computes one 16x16 output tile
    // Warp ID within the grid (each warp is an independent unit)
    int warpM = (blockIdx.x * blockDim.x + threadIdx.x) / 32;
    int warpN = (blockIdx.y * blockDim.y + threadIdx.y);

    // Calculate the starting row and column for this warp's output tile
    int warp_row = warpM * WMMA_M;
    int warp_col = warpN * WMMA_N;

    // Bounds check
    if (warp_row >= M || warp_col >= N)
        return;

    // Declare the fragments
    wmma::fragment<wmma::matrix_a, WMMA_M, WMMA_N, WMMA_K, half, wmma::row_major> a_frag;
    wmma::fragment<wmma::matrix_b, WMMA_M, WMMA_N, WMMA_K, half, wmma::row_major> b_frag;
    wmma::fragment<wmma::accumulator, WMMA_M, WMMA_N, WMMA_K, float> acc_frag;

    // Initialize accumulator to zero
    wmma::fill_fragment(acc_frag, 0.0f);

    // Loop over K dimension in steps of WMMA_K
    for (int i = 0; i < K; i += WMMA_K) {
        if (i < K) {
            // Step 1: Load the inputs into fragments
            wmma::load_matrix_sync(a_frag, A + warp_row * lda + i, lda);
            wmma::load_matrix_sync(b_frag, B + i * ldb + warp_col, ldb);

            // Step 2: Perform the matrix multiplication
            wmma::mma_sync(acc_frag, a_frag, b_frag, acc_frag);
        }
    }

    // Step 3: Store the output
    wmma::store_matrix_sync(C + warp_row * ldc + warp_col, acc_frag, ldc, wmma::mem_row_major);
}

// Convert float to double
__global__ void float_to_double(const float *input, double *output, int size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < size) {
        output[idx] = (double)input[idx];
    }
}

// CPU reference implementation
void cpu_gemm(const double *A, const double *B, double *C, int M, int K, int N) {
    for (int i = 0; i < M; i++) {
        for (int j = 0; j < N; j++) {
            double sum = 0.0;
            for (int k = 0; k < K; k++) {
                sum += A[i * K + k] * B[k * N + j];
            }
            C[i * N + j] = sum;
        }
    }
}

// Calculate relative error
double calculate_error(const double *ref, const double *result, int size) {
    double max_error = 0.0;
    double sum_error = 0.0;

    for (int i = 0; i < size; i++) {
        double error = fabs(ref[i] - result[i]);
        double rel_error = error / (fabs(ref[i]) + 1e-10);
        max_error = fmax(max_error, rel_error);
        sum_error += rel_error;
    }

    return sum_error / size;
}

void print_wmma_info(int M, int K, int N) {
    printf("\n=== WMMA Configuration ===\n");
    printf("Fragment dimensions: M=%d, N=%d, K=%d\n", WMMA_M, WMMA_N, WMMA_K);

    // Calculate number of fragments needed
    int frags_M = (M + WMMA_M - 1) / WMMA_M;
    int frags_N = (N + WMMA_N - 1) / WMMA_N;
    int frags_K = (K + WMMA_K - 1) / WMMA_K;

    printf("Number of fragments:\n");
    printf("  M direction: %d fragments (covering %d elements)\n", frags_M, M);
    printf("  N direction: %d fragments (covering %d elements)\n", frags_N, N);
    printf("  K direction: %d fragments (covering %d elements)\n", frags_K, K);
    printf("  Total fragments used: %d\n", frags_M * frags_N * frags_K);
    printf("  Fragment type: half precision (FP16) input, float accumulator\n");
    printf("==========================\n\n");
}

int main(int argc, char **argv) {
    if (argc < 2) {
        printf("Usage: %s <size> or %s <M> <K> <N>\n", argv[0], argv[0]);
        printf("Example: %s 1024 2048 1024  (for A: 1024x2048, B: 2048x1024)\n", argv[0]);
        return 1;
    }

    int M, K, N;
    if (argc == 2) {
        // Square matrices
        M = K = N = atoi(argv[1]);
    } else if (argc == 4) {
        M = atoi(argv[1]);
        K = atoi(argv[2]);
        N = atoi(argv[3]);
    } else {
        fprintf(stderr, "Invalid arguments\n");
        return 1;
    }

    printf("Matrix dimensions: A(%d x %d) * B(%d x %d) = C(%d x %d)\n\n",
           M, K, K, N, M, N);

    // Print WMMA information
    print_wmma_info(M, K, N);

    // Allocate host memory
    size_t bytes_A = M * K * sizeof(double);
    size_t bytes_B = K * N * sizeof(double);
    size_t bytes_C = M * N * sizeof(double);

    double *h_A = (double*)malloc(bytes_A);
    double *h_B = (double*)malloc(bytes_B);
    double *h_C_cpu = (double*)malloc(bytes_C);
    double *h_C_gemm = (double*)malloc(bytes_C);
    double *h_C_wmma = (double*)malloc(bytes_C);

    // Initialize matrices
    for (int i = 0; i < M * K; i++) {
        h_A[i] = (double)(rand() % 100) / 10.0;
    }
    for (int i = 0; i < K * N; i++) {
        h_B[i] = (double)(rand() % 100) / 10.0;
    }

    // CPU computation
    printf("Running CPU reference...\n");
    auto cpu_start = std::chrono::high_resolution_clock::now();
    cpu_gemm(h_A, h_B, h_C_cpu, M, K, N);
    auto cpu_end = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> cpu_duration = cpu_end - cpu_start;
    printf("CPU time: %.3f ms\n\n", cpu_duration.count() * 1000.0);

    // Allocate device memory for double precision
    double *d_A, *d_B, *d_C_gemm;
    CHECK(cudaMalloc(&d_A, bytes_A));
    CHECK(cudaMalloc(&d_B, bytes_B));
    CHECK(cudaMalloc(&d_C_gemm, bytes_C));

    CHECK(cudaMemcpy(d_A, h_A, bytes_A, cudaMemcpyHostToDevice));
    CHECK(cudaMemcpy(d_B, h_B, bytes_B, cudaMemcpyHostToDevice));

    // Run tiled GEMM kernel
    printf("Running tiled GEMM kernel...\n");
    dim3 block_gemm(TILE_SIZE, TILE_SIZE);
    dim3 grid_gemm((N + TILE_SIZE - 1) / TILE_SIZE,
                   (M + TILE_SIZE - 1) / TILE_SIZE);

    cudaEvent_t start, stop;
    CHECK(cudaEventCreate(&start));
    CHECK(cudaEventCreate(&stop));

    CHECK(cudaEventRecord(start));
    gemm<<<grid_gemm, block_gemm>>>(d_A, d_B, d_C_gemm, M, K, N);
    CHECK(cudaEventRecord(stop));
    CHECK(cudaEventSynchronize(stop));

    float gemm_time = 0;
    CHECK(cudaEventElapsedTime(&gemm_time, start, stop));
    CHECK(cudaMemcpy(h_C_gemm, d_C_gemm, bytes_C, cudaMemcpyDeviceToHost));

    double gemm_error = calculate_error(h_C_cpu, h_C_gemm, M * N);
    printf("GEMM time: %.3f ms\n", gemm_time);
    printf("GEMM average relative error: %.10f\n\n", gemm_error);

    // Allocate device memory for half precision (for WMMA)
    half *d_A_half, *d_B_half;
    float *d_C_wmma_float;  // WMMA outputs float
    double *d_C_wmma_double;  // For final conversion to double
    CHECK(cudaMalloc(&d_A_half, M * K * sizeof(half)));
    CHECK(cudaMalloc(&d_B_half, K * N * sizeof(half)));
    CHECK(cudaMalloc(&d_C_wmma_float, M * N * sizeof(float)));
    CHECK(cudaMalloc(&d_C_wmma_double, bytes_C));

    // Convert double to half
    int threads = 256;
    int blocks_A = (M * K + threads - 1) / threads;
    int blocks_B = (K * N + threads - 1) / threads;

    double_to_half<<<blocks_A, threads>>>(d_A, d_A_half, M * K);
    double_to_half<<<blocks_B, threads>>>(d_B, d_B_half, K * N);
    CHECK(cudaDeviceSynchronize());

    // Run WMMA kernel
    printf("Running WMMA Tensor Core kernel...\n");

    // For WMMA: each warp (32 threads) computes one 16x16 output tile
    // Use 128 threads per block (4 warps) for good occupancy
    // Block dimensions: 128 threads in X (4 warps), 1 in Y
    int warps_per_block_x = 8;  // 4 warps per block
    dim3 block_wmma(warps_per_block_x * 32, 1);  // 128 threads in X, 1 in Y
    dim3 grid_wmma(((M + WMMA_M - 1) / WMMA_M + warps_per_block_x - 1) / warps_per_block_x,
                   (N + WMMA_N - 1) / WMMA_N);

    printf("WMMA Grid: (%d, %d), Block: (%d, %d)\n",
           grid_wmma.x, grid_wmma.y, block_wmma.x, block_wmma.y);

    CHECK(cudaEventRecord(start));
    wmma_gemm<<<grid_wmma, block_wmma>>>(d_A_half, d_B_half, d_C_wmma_float, M, K, N);
    CHECK(cudaGetLastError());

    // Convert float output to double for comparison
    int blocks_C = (M * N + threads - 1) / threads;
    float_to_double<<<blocks_C, threads>>>(d_C_wmma_float, d_C_wmma_double, M * N);
    CHECK(cudaGetLastError());

    CHECK(cudaEventRecord(stop));
    CHECK(cudaEventSynchronize(stop));

    float wmma_time = 0;
    CHECK(cudaEventElapsedTime(&wmma_time, start, stop));
    CHECK(cudaMemcpy(h_C_wmma, d_C_wmma_double, bytes_C, cudaMemcpyDeviceToHost));

    double wmma_error = calculate_error(h_C_cpu, h_C_wmma, M * N);
    printf("WMMA time: %.3f ms\n", wmma_time);
    printf("WMMA average relative error: %.10f\n", wmma_error);

    // Debug: Print first few values
    printf("DEBUG - First 10 values:\n");
    printf("  CPU:  ");
    for (int i = 0; i < 10 && i < M * N; i++) printf("%.2f ", h_C_cpu[i]);
    printf("\n  GEMM: ");
    for (int i = 0; i < 10 && i < M * N; i++) printf("%.2f ", h_C_gemm[i]);
    printf("\n  WMMA: ");
    for (int i = 0; i < 10 && i < M * N; i++) printf("%.2f ", h_C_wmma[i]);
    printf("\n\n");

    // Print performance comparison
    printf("=== Performance Summary ===\n");
    printf("CPU:  %.3f ms\n", cpu_duration.count() * 1000.0);
    printf("GEMM: %.3f ms (%.2fx speedup over CPU)\n",
           gemm_time, cpu_duration.count() * 1000.0 / gemm_time);
    printf("WMMA: %.3f ms (%.2fx speedup over CPU, %.2fx over GEMM)\n",
           wmma_time, cpu_duration.count() * 1000.0 / wmma_time, gemm_time / wmma_time);
    printf("\n=== Accuracy Summary ===\n");
    printf("GEMM avg relative error: %.10f\n", gemm_error);
    printf("WMMA avg relative error: %.10f\n", wmma_error);

    // Cleanup
    CHECK(cudaEventDestroy(start));
    CHECK(cudaEventDestroy(stop));
    free(h_A);
    free(h_B);
    free(h_C_cpu);
    free(h_C_gemm);
    free(h_C_wmma);
    CHECK(cudaFree(d_A));
    CHECK(cudaFree(d_B));
    CHECK(cudaFree(d_C_gemm));
    CHECK(cudaFree(d_A_half));
    CHECK(cudaFree(d_B_half));
    CHECK(cudaFree(d_C_wmma_float));
    CHECK(cudaFree(d_C_wmma_double));

    return 0;
}


Writing wmma_matrixMul8.cu
